# TOÁN ỨNG DỤNG VÀ THỐNG KÊ

## Đồ án Linear Regression

---

**Thông Tin Sinh Viên**
- Họ tên: Trương Tiến Anh
- MSSV: 22120017
- Lớp: Chiều thứ 7

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import linear_model


# Xây dựng các hàm hỗ trợ

In [7]:
# Linear regression
def linear_regression(features, target, fit_intercept=True):
    try:
        model = linear_model.LinearRegression(fit_intercept=fit_intercept).fit(features, target)
        return model.coef_, model.intercept_
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [6]:
# Cross Validation
def cross_validation(features, target, k_folds):
    kf = KFold(n_splits=k_folds)
    error_list = []

    for train_indices, test_indices in kf.split(features):
        features_train, features_test = features[train_indices], features[test_indices]
        target_train, target_test = target[train_indices], target[test_indices]
        coefficients, intercept = linear_regression(features_train, target_train)
        
        if coefficients is None or intercept is None:
            print("An error occurred during model training.")
            return None
        
        predictions = np.dot(features_test, coefficients) + intercept
        error = np.mean(np.abs(target_test - predictions))
        error_list.append(error)

    return np.mean(error_list)

In [3]:
# Đọc file CSV
df = pd.read_csv('wine.csv')

# Chuẩn bị dữ liệu
features = df.iloc[:, 1:].to_numpy()  # Sử dụng tất cả các cột ngoại trừ cột đầu tiên
target = df.iloc[:, 0].to_numpy()     # Sử dụng cột đầu tiên làm nhãn
labels = df.columns[1:]

print("Features (Đặc trưng):")
print(features)
print("\nTarget (Nhãn):")
print(target)
print("\nLabels (Tên các cột đặc trưng):")
print(labels)

Features (Đặc trưng):
[[ 7.4   0.7   0.   ...  3.51  0.56  9.4 ]
 [ 7.8   0.88  0.   ...  3.2   0.68  9.8 ]
 [ 7.8   0.76  0.04 ...  3.26  0.65  9.8 ]
 ...
 [ 7.9   0.58  0.23 ...  3.21  0.58  9.5 ]
 [ 7.7   0.57  0.21 ...  3.16  0.54  9.8 ]
 [ 7.7   0.26  0.26 ...  3.15  0.79 10.9 ]]

Target (Nhãn):
[5 5 5 ... 6 6 6]

Labels (Tên các cột đặc trưng):
Index(['"fixed acidity"', '"volatile acidity"', '"citric acid"',
       '"residual sugar"', '"chlorides"', '"free sulfur dioxide"',
       '"total sulfur dioxide"', '"density"', '"pH"', '"sulphates"',
       '"alcohol"'],
      dtype='object')


### **Ý tưởng**
### **Linear Regression**

##### Giới thiệu

* **Xây dựng mô hình đơn giản theo công thức: $Ax = b$**
  * $A$ là ma trận dữ liệu (các tính chất của rượu)
  * $b$ là nhãn (label) của mỗi dòng dữ liệu trong ma trận $A$, được thể hiện dưới dạng đánh giá (xếp loại) rượu

##### Công thức mô hình

* **Với mô hình trên, ta có thể tìm mô hình theo công thức:**
  $\hat{x}=A^\dagger\cdot b$
  * Khi đó, mô hình sẽ đi qua gốc tọa độ của đồ thị, điều này dẫn đến sự hạn chế.
  * Do đó, chúng ta chọn mô hình theo công thức sau:
  \[Ax + b_0 = b\]
  * Với công thức này, mô hình có thể dịch chuyển trên trục tung, tăng sự linh hoạt.

##### Sử dụng `sklearn` để xây dựng mô hình

* Bộ thư viện `sklearn` cung cấp công cụ để dựng mô hình hồi quy tuyến tính với công thức đã chọn:
  * Sau khi fit các tham số bao gồm \(A\) và \(b\) vào mô hình, các thuộc tính quan trọng của mô hình Linear Regression được sinh ra, bao gồm:
    * `coef_`: \(\hat{x}\)
    * `intercept_`: \(b_0\)
  * Gán các giá trị này vào công thức, ta được mô hình hồi quy tuyến tính cần tìm.


### **Cross Validation**

##### Giới thiệu

* **K-Fold Cross-validation** là một kỹ thuật dùng để đánh giá hiệu quả của mô hình trên tập dữ liệu.
  * **Xáo trộn dữ liệu** (tùy chọn)
  * **Chia dataset thành k nhóm** (folds)
  * **Quy trình thực hiện với mỗi nhóm**:
    1. Sử dụng nhóm hiện tại để đánh giá hiệu quả mô hình.
    2. Các nhóm còn lại được sử dụng để huấn luyện mô hình.
    3. Huấn luyện mô hình trên tập train.
    4. Đánh giá mô hình trên tập test.
  * **Tổng hợp hiệu quả qua các đánh giá**.
##### Sử dụng `sklearn` cho K-Fold Cross-validation

* Bộ thư viện `sklearn` cung cấp hàm `sklearn.model_selection.KFold` để tự động chia tập dữ liệu thành \(k\) nhóm và `split` thành các bộ dữ liệu train/test khác nhau.
  * **Quy trình chi tiết**:
    1. **Xây dựng mô hình** trên tập train, ta được các tham số $\hat{x}$, $b_0$
    2. **Áp dụng mô hình lên tập test**: $A_{test}\cdot \hat{x}=b'$
    3. **Tính sai số** so với nhãn của tập test: $|b'-b_{test}|$. Kết quả này là một ma trận có shape giống  $b_{test}$. Tính trung bình của ma trận này để được sai số của mô hình trên tập train/test đó.
    4. **Lặp lại cho tất cả các tập train/test** được split ra ở trên, sau đó tính trung bình các sai số này để được sai số trung bình của mô hình dựa trên phương pháp Cross Validation.
  * **Số \(k\) được chọn cho K-Fold thường là 10**.
### Đọc File CSV
* **Tách dữ liệu đọc từ `wine.csv` thành bộ dữ liệu và bộ label**

# Xây dựng mô hình đánh giá chất lượng rượu sử dụng phương pháp hồi quy tuyến tính

### Câu a. Sử dụng bộ 11 đặc trưng đề bài cung cấp.

In [10]:

coefficients, intercept = linear_regression(features, target)
cv_error = cross_validation(features, target, k_folds=10)

# In kết quả
coefficients_str = "\n".join([f"Feature {i+1}: {coef:.6f}" for i, coef in enumerate(coefficients)])
print("Model Coefficients:\n" + coefficients_str)
print(f"Intercept: {intercept:.6f}")
print(f"Cross-Validation Error (10-fold): {cv_error:.6f}")

Model Coefficients:
Feature 1: 0.047525
Feature 2: -1.068743
Feature 3: -0.268711
Feature 4: 0.034974
Feature 5: -1.597296
Feature 6: 0.003488
Feature 7: -0.003798
Feature 8: -39.469081
Feature 9: -0.245576
Feature 10: 0.773841
Feature 11: 0.269377
Intercept: 42.917162
Cross-Validation Error (10-fold): 0.509451


### Ý tưởng
* Dùng `linear_regression()` đã cài đặt để tìm model trên tất cả các tính chất

### Câu b. Sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất. (Dùng phương pháp Cross Validation)

In [3]:
# Choose best property
def choose_best_property(X, y):
    cross_val_errors = []

    for feature_index in range(X.shape[1]):
        cross_val_error = cross_validation(X[:, feature_index:feature_index + 1], y, k_folds=10)
        if cross_val_error is not None:
            cross_val_errors.append(cross_val_error)
        else:
            cross_val_errors.append(np.inf)  # Nếu có lỗi, gán giá trị lỗi rất lớn để không chọn đặc trưng này
    
    best_property_index = np.argmin(cross_val_errors)
    return cross_val_errors, best_property_index

In [38]:
error_list, best_index = choose_best_property(features, target)

# Xây dựng mô hình với tính chất tốt nhất
best_feature = features[:, best_index:best_index + 1]
coefficients, intercept = linear_regression(best_feature, target)

# In kết quả
print("\n=== Best Property Selection ===")
print(f"Best property: {labels[best_index]}")
print("\n=== Model ===")
print(f"Model: y = {coefficients[0]:.6f} * x + {intercept:.6f}")
print("\n=== Cross-Validation Error ===")
print(f"Cross-Validation Error: {error_list[best_index]:.6f}\n")


=== Best Property Selection ===
Best property: "alcohol"

=== Model ===
Model: y = 0.374710 * x + 1.774076

=== Cross-Validation Error ===
Cross-Validation Error: 0.568944



### Ý tưởng
* Đối với từng cột (tính chất), chạy hàm `cross_validation()` để tìm sai số của mô hình dựa trên mỗi tính chất.

* Tìm tính chất có sai số bé nhất để xác định tính chất tốt nhất.

* Chạy `linear_regression()` để tìm mô hình dựa trên tính chất tốt nhất này.

### Câu c. Xây dựng một mô hình của riêng bạn cho kết quả tốt nhất

In [4]:
# Build model
def build_model(X, y, error_list, max_features):
    sorted_error_indices = np.argsort(error_list)
    full_error = cross_validation(X, y, k_folds=10)

    abs_error_list = []
    cv_error_list = []
    selected_features_list = []

    for num_features in range(2, max_features - 1):
        selected_features = sorted_error_indices[:num_features]
        X_selected = X[:, selected_features]
        cv_error = cross_validation(X_selected, y, k_folds=10)
        
        if cv_error is not None:
            cv_error_list.append(cv_error)
            abs_error_list.append(np.abs(full_error - cv_error))
            selected_features_list.append(selected_features)
        else:
            cv_error_list.append(np.inf)
            abs_error_list.append(np.inf)
            selected_features_list.append(selected_features)
    
    best_model_index = np.argmin(abs_error_list)
    return cv_error_list, selected_features_list, best_model_index

In [40]:
cv_error_list, prop_list, best_index_prop = build_model(features, target, error_list, len(labels))

# Xây dựng mô hình với top x tính chất tốt nhất
best_properties = features[:, prop_list[best_index_prop]]
coefficients, intercept = linear_regression(best_properties, target)

# In kết quả
print("\n=== Model Building with Top Features ===")
print("Best properties:")
for i, prop in enumerate(labels[prop_list[best_index_prop]], start=1):
    print(f"{i}. {prop}")

print("\nModel:")
print(f"A{coefficients} + {intercept} = b")

print("\nCross-Validation Error:", cv_error_list[best_index_prop])


=== Model Building with Top Features ===
Best properties:
1. "alcohol"
2. "volatile acidity"
3. "total sulfur dioxide"
4. "citric acid"
5. "sulphates"
6. "density"
7. "fixed acidity"
8. "chlorides"
9. "free sulfur dioxide"

Model:
A[ 2.79228621e-01 -1.08519171e+00 -3.27609434e-03 -2.50067753e-01
  7.55341209e-01 -3.10768383e+01  5.89618605e-02 -1.44151674e+00
  2.85670572e-03] + 33.61492533087147 = b

Cross-Validation Error: 0.5104161230846669


### Ý tưởng
* Chọn ra n tính chất tốt nhất (cho n chạy từ 2 đến 10) rồi chạy `cross_validation()`

* Chọn ra bộ tính chất tốt nhất (sai số thấp nhất), các tính chất sẽ được chọn để dựng mô hình

* Chạy `linear_regression()` để tìm model dựa trên các tính chất này

* Kết quả của mô hình tự xây dựng và của mô hình 11 tính chất:

    * Mô hình tự xây dựng: 0.5104161230846669
    * Mô hình 11 tính chất: 0.5094507964775307